In [ ]:
!pip install transformers==3

In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# df_train = pd.read_csv('train.csv', header = None)
df_train = pd.read_csv('balanced_dataset.csv', header = None)

df_train.columns = ['label', 'content', 'index']

# df_train_set, df_valid_set = train_test_split(df_train, test_size = 0.2, random_state = 42)

df_train_set, df_test_set = train_test_split(df_train, test_size = 0.2, random_state = 42)

# sns.countplot(df.rating)

class_name = ['True', 'False', 'Unindentified']

In [ ]:
df_train_set.all()

label      True
content    True
index      True
dtype: bool

In [ ]:
df_test_set

,label,content,index
2186,F,Cow dung has many benefits. I think it can kil...,1737
3351,U,Goldman Sachs expects coronavirus outbreak to ...,1106
2651,F,A claim on Facebook that began to circulate ar...,3231
2104,F,US President Donald Trump announced that Swiss...,1655
3811,U,@CDCgov @DrNancyM_CDC Why? Stop the flights! T...,1900
...,...,...,...
1479,T,Remember when Ford cut a billion dollars from ...,5640
2534,F,Koreans who were on the cruiseship Diamond Pr...,3113
2029,F,nations including USA and UK wants NARENDER...,1580
2266,F,A message claims that the pH value of the nove...,1817


In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
MAX_LEN = 50

In [ ]:
df_train.dropna()

,label,content,index
0,label,content,index
1,T,Almost people in Italy died from the coron...,4
2,T,A doctor in Italy warned coronavirus wasn t ju...,5
3,T,Washing your hands decreases the number of mic...,20
4,T,The Chinese government is building a hospital ...,21
...,...,...,...
4856,U,"Kudlow is correct here. Like SARS and Ebola, c...",7156
4857,U,@muhdaimanharith @dytiaaurinh @DGHisham A pati...,7157
4858,U,The authorities have shifted their policy from...,7158
4859,U,'New technology and better coordination have s...,7159


In [ ]:
type(df_train.content[65])

str

In [ ]:
def output_bert_vector(document):

  return tokenizer.encode_plus(
                    # string
                    document,
                    # Whether or not to encode the sequences with the special tokens relative to their model.
                    add_special_tokens = True,
                    # Controls the maximum length to use by one of the truncation/padding parameters.
                    max_length = MAX_LEN,
                    # These require two different sequences to be joined in a 
                    # single “input_ids” entry, which usually is performed with the help of special tokens, 
                    # such as the classifier ([CLS]) and separator ([SEP]) tokens.
                    return_token_type_ids = False,
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    # If set, will return tensors instead of list of python integers.
                    # 'pt': Return PyTorch torch.Tensor objects.
                    # return_tensors = 'pt'
                    truncation=True
                    )['input_ids']

In [ ]:
len(output_bert_vector("hello world."))

50

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental. please. okay."))

[1274, 1204, 1129, 1177, 9228, 1348, 119, 4268, 119, 3008, 119]

In [ ]:
i = 0
for content in df_train['content']:
  if content == 'content':
    continue
  print(len(content.split()))

16
29
19
15
13
20
12
28
29
18
18
28
19
7
37
11
18
27
21
18
22
33
21
10
31
14
21
63
32
24
23
22
12
32
21
29
25
25
14
48
31
37
35
29
9
15
8
13
42
32
50
9
93
28
8
11
14
10
8
9
8
26
29
34
33
26
22
13
77
15
8
21
40
14
23
17
25
17
12
42
61
12
20
25
12
15
22
20
10
31
18
21
13
11
24
18
28
25
12
15
11
26
21
19
12
22
23
13
13
25
21
17
13
17
17
13
6
11
25
35
20
10
36
17
22
11
31
12
43
37
9
13
25
11
30
23
16
13
51
9
23
13
33
21
13
37
38
9
45
11
29
13
14
33
16
15
13
17
21
25
10
16
34
28
23
13
25
48
32
19
41
27
40
33
27
15
13
34
29
37
45
55
27
14
34
31
15
50
27
18
44
39
17
30
31
11
30
14
24
12
47
11
15
31
18
26
24
12
41
11
30
39
15
11
39
10
11
10
24
20
18
15
37
31
30
20
12
22
38
21
39
12
19
17
49
59
36
49
36
54
41
30
112
99
52
43
43
106
61
60
36
23
18
31
39
29
41
11
9
17
20
39
18
36
8
30
11
9
10
36
16
45
15
11
43
19
19
23
29
17
11
37
34
41
25
15
24
24
27
14
44
14
15
33
8
10
16
25
7
16
38
24
20
10
29
12
36
32
29
15
30
20
35
33
22
32
8
11
33
39
22
14
15
11
18
9
10
56
35
31
35
16
8
8
28
12
24
30
12
27


In [ ]:
df_train_array = []
for content in df_train_set['content']:
  if content == 'content':
    continue
  df_train_array.append(output_bert_vector(content))

In [ ]:
print(len(df_train_array[0]))

50


In [ ]:
labels_for_train = []
for label in df_train_set['label']:
  if label == 'label':
    continue
  elif label == 'T':
    labels_for_train.append(0)
  elif label == 'F':
    labels_for_train.append(1)
  elif label == 'U':
    labels_for_train.append(2)

In [ ]:
len(labels_for_train)

3887

In [ ]:
embedding_dim=64

In [ ]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=embedding_dim),
    # tf.keras.layers.LSTM(128),
   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    # tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          1855744   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 1,880,771
Trainable params: 1,880,771
Non-trainable params: 0
_________________________________________________________________


In [ ]:
l = labels_for_train
print(set(l))

{0, 1, 2}


In [ ]:
np.array(df_train_array).shape

(3887, 50)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(np.array(df_train_array), np.array(labels_for_train), epochs=num_epochs, verbose="auto")

Epoch 1/10
122/122 [==============================] - 11s 60ms/step - loss: 0.9651 - accuracy: 0.5058
Epoch 2/10
122/122 [==============================] - 7s 59ms/step - loss: 0.6803 - accuracy: 0.6571
Epoch 3/10
122/122 [==============================] - 7s 60ms/step - loss: 0.4538 - accuracy: 0.8032
Epoch 4/10
122/122 [==============================] - 7s 60ms/step - loss: 0.2231 - accuracy: 0.9143
Epoch 5/10
122/122 [==============================] - 7s 59ms/step - loss: 0.1046 - accuracy: 0.9673
Epoch 6/10
122/122 [==============================] - 7s 59ms/step - loss: 0.0600 - accuracy: 0.9820
Epoch 7/10
122/122 [==============================] - 7s 59ms/step - loss: 0.0287 - accuracy: 0.9923
Epoch 8/10
122/122 [==============================] - 7s 59ms/step - loss: 0.0184 - accuracy: 0.9954
Epoch 9/10
122/122 [==============================] - 7s 60ms/step - loss: 0.0169 - accuracy: 0.9956
Epoch 10/10
122/122 [==============================] - 7s 59ms/step - loss: 0.0096 - accur

In [ ]:
print(df_train.shape)
i = 3
print(df_train.content[i], df_train.label[i])
model.predict([output_bert_vector(df_train.content[i])])
# df_train.content[i]

(4861, 3)
Washing your hands decreases the number of microbes on your hands and helps prevent the spread of infectious diseases T


array([[9.9864095e-01, 2.6221806e-04, 1.0967506e-03]], dtype=float32)

In [ ]:
!pip install pyyaml h5py

In [ ]:
model.save('my_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')

In [ ]:
new_model.get_weights()

In [ ]:
new_model.summary()

In [ ]:
loaded_model = tf.keras.models.load_model('.')


In [ ]:
type(loaded_model)

In [ ]:
x = model.layers[-1].output 

In [ ]:
x = model.layers
x

In [ ]:
new_model = tf.keras.Sequential()
for layer in loaded_model.layers[:-1]: # go through until last layer
    new_model.add(layer)

In [ ]:
new_model.layers

In [ ]:
old_weights = loaded_model.get_weights()[i]

In [ ]:
x = tf.keras.layers.Dense(256)(x)
# predictions = Dense(15, activation = "softmax")(x)

In [ ]:
x

In [ ]:
i = 60
print(df_train_set.content[i], df_train_set.label[i])
emb = model.predict([output_bert_vector(df_train_set.content[i])])
print(emb, emb.shape)

In [ ]:
new_model.save('./my_model.pb')

In [ ]:
from google.colab import files
files.download("/content/my_model.pb")

In [ ]:
!zip -r model.zip my_model.pb

In [ ]:
model.save('old_model')

In [ ]:
!zip -r old_model.zip old_model